# Transcript data feature engineering

### Loading Packages

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

2022-10-27 02:06:54.651368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 02:06:54.789794: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-27 02:06:55.483455: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


['/device:CPU:0', '/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


2022-10-27 02:06:56.429443: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-27 02:07:01.063416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 38407 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-10-27 02:07:01.065425: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 38407 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2022-10-27 02:07:01.067135: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:2 with 38407 MB memory:  -> device: 2, name: A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capab

In [4]:
#import os
#os.environ['CUDA_VISIBLE_DEVICES']

In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import time
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading datasets

In [9]:
%cd ..

/workspace/asurion_f22_muby


In [10]:
%cd fe_data_new

/workspace/asurion_f22_muby/fe_data_new


In [11]:
%ls

Untitled.ipynb                            total_collated_transcripts.csv
accepted_upsale_collated_transcripts.csv  total_encoded_standardized.csv
accepted_upsale_encoded_tabular.csv       total_encoded_tabular.csv
accepted_upsale_merged_asurion.csv        total_merged_asurion.csv
tabular_april_encoded.csv                 transcript_april_collated_encoded.csv
tabular_july_encoded.csv                  transcript_july_collated_encoded.csv
tabular_june_encoded.csv                  transcript_june_collated_encoded.csv
tabular_march_encoded.csv                 transcript_march_collated_encoded.csv
tabular_may_encoded.csv                   transcript_may_collated_encoded.csv


In [12]:
%cd ..

/workspace/asurion_f22_muby


In [13]:
transcript = pd.read_csv("fe_data_new/total_collated_transcripts.csv")

In [14]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262644 entries, 0 to 1262643
Data columns (total 5 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   id                                1262644 non-null  object
 1   sales_offer_date                  1262644 non-null  object
 2   accepted_flg                      1262644 non-null  bool  
 3   encrypted_collated_transcription  1258390 non-null  object
 4   month                             1262644 non-null  object
dtypes: bool(1), object(4)
memory usage: 39.7+ MB


In [15]:
transcript["month"].value_counts()

june     283795
may      266497
march    250713
april    243426
july     218213
Name: month, dtype: int64

### Splitting data into training set and hold-out set

In [16]:
transcript_train = transcript[transcript["month"] != "july"]
transcript_test = transcript[transcript["month"] == "july"]

In [17]:
#transcript_train.to_csv("fe_data_v2/transcript_data/transcript_train_data.csv", index = False)

In [18]:
#transcript_test.to_csv("fe_data_v2/transcript_data/transcript_test_data.csv", index = False)

### Feature Engineering on training set

In [19]:
transcript_train.head()

,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march


In [20]:
from torch import nn
import torch

In [21]:
transcript_train["encrypted_collated_transcription"] = transcript_train["encrypted_collated_transcription"].astype(str)
transcript_test["encrypted_collated_transcription"] = transcript_test["encrypted_collated_transcription"].astype(str)

/tmp/ipykernel_3188/455747962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_train["encrypted_collated_transcription"] = transcript_train["encrypted_collated_transcription"].astype(str)
/tmp/ipykernel_3188/455747962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_test["encrypted_collated_transcription"] = transcript_test["encrypted_collated_transcription"].astype(str)


In [22]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3,4,5,6,7"

In [23]:
from transformers import pipeline
pega = pipeline("summarization", model="sshleifer/distilbart-xsum-12-6")

In [24]:
pipe_out_pega = pega(transcript_train.iloc[8,3][:3000])
summary = pipe_out_pega[0]["summary_text"].replace(" .<n>", ". ")
len(summary)

83

#### Partitioning convo function for training and testing set

In [25]:
convo = transcript_train.iloc[8,3]
convo_rev = convo[::-1]
index = convo_rev.find(".")
convo_rev[:index][::-1]

' Other'

In [34]:
def transform_sentence(sentence, k):
    length_sentence = len(sentence)
    sentences = ""
    list_of_sentence = []
    count = 0
    m = 0
    n = m + k
    while count < length_sentence:
        sentence_ext = sentence[m:n]
        sentence_rev = sentence_ext[::-1]
        rev_index = sentence_rev.find(".")
        sentence_ext = sentence_rev[rev_index:][::-1]
        list_of_sentence.append(sentence_ext)
        sentences += sentence_ext
        m = len(sentences)
        if len(sentences) < length_sentence:
            n = m + k
        else:
            n = length_sentence
        
        count += len(sentence_ext)
    
    #lengths = []
    new_list = []
    for sentence_run in list_of_sentence:
        if len(sentence_run) > 1:
            new_list.append(sentence_run)
            #lengths.append(len(sentence))
    
    sentence_reversed = sentence[::-1]
    reversed_index = sentence_reversed.find(".")
    sentence_final = sentence_reversed[:reversed_index][::-1]
    
    if len(new_list) > 0:
        new_list[-1] = new_list[-1] + sentence_final
    
    return new_list

In [35]:
transform_sentence(convo, 2000)

["Thank you for calling [CLIENT] Tech Coach. My name is [NAME]. May I have your first and last name, please? Ok. That's Felicia moore. Ok. Can I also have the phone number you're calling about? Yes. Five [NUMBER] [NUMBER]. Two [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER] [NUMBER]. Thank you. How can I help you today? Yes. I'm calling because I am having complaints no phone calls coming through. my account keep dropping. I see. Thank you so much for giving us a call in for patiently waiting on the line to be assisted. Let me go ahead and pull up the account. So we'll be able to resolve this problem. what's your account, pin, Felicia? * Thank you. Let me just quickly pull the account up Since when did it start to happen, I'm sorry. I'm bleeding, uh, today, I guess. Ok. people are complaining that they've been calling me, I do. Ok. So people cannot And what I've tried to make a outgoing call. Yeah. When I try to make apple outgoing calls they're saying that reach. Mhmm. The calls are not 

In [28]:
import numpy as np

In [29]:
transcript_train.head()

,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march


In [30]:
def partition(df):
    df["length"] = df.iloc[:,3].str.len()
    lengths = list(df["length"])
    df["length_3rd"] = df["length"]//3
    df["length_3rd"] = df["length_3rd"].astype(int)
    len_3rd = list(df["length_3rd"])
    transcript_convo = list(df.iloc[:,3])
    transcript_convo_first_3rd = []
    transcript_convo_second_3rd = []
    transcript_convo_third_3rd = []
    transcript_convo_transform = []
    for i in tqdm(range(len(transcript_convo))):
        transcript_convo_first_3rd.append(transform)
        transcript_convo_second_3rd.append(transcript_convo[i][len_3rd[i]:len_3rd[i]*2])
        transcript_convo_third_3rd.append(transcript_convo[i][len_3rd[i]*2:])
    df["first_3rd"] = transcript_convo_first_3rd
    df["second_3rd"] = transcript_convo_second_3rd
    df["third_3rd"] = transcript_convo_third_3rd

In [31]:
def new_partition(df):
    df["length"] = df.iloc[:,3].str.len()
    #lengths = list(df["length"])
    #df["length_3rd"] = df["length"]//3
    #df["length_3rd"] = df["length_3rd"].astype(int)
    #len_3rd = list(df["length_3rd"])
    transcript_convo = list(df.iloc[:,3])
    #transcript_convo_first_3rd = []
    #transcript_convo_second_3rd = []
    #transcript_convo_third_3rd = []
    transcript_convo_transform = []
    for i in tqdm(range(len(transcript_convo))):
        transcript_convo_transform.append(transform_sentence(transcript_convo[i], 2000))
    df["transcript_chunked"] = transcript_convo_transform

In [36]:
new_partition(transcript_train)

/tmp/ipykernel_3188/3442767910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["length"] = df.iloc[:,3].str.len()
100%|██████████████████████████████████████████████████████████████████████| 1044431/1044431 [00:43<00:00, 24194.05it/s]
/tmp/ipykernel_3188/3442767910.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["transcript_chunked"] = transcript_convo_transform


In [37]:
new_partition(transcript_test)

/tmp/ipykernel_3188/3442767910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["length"] = df.iloc[:,3].str.len()
100%|████████████████████████████████████████████████████████████████████████| 218213/218213 [00:09<00:00, 22876.10it/s]
/tmp/ipykernel_3188/3442767910.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["transcript_chunked"] = transcript_convo_transform


In [38]:
transcript_train.head()

,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month,length,transcript_chunked
0,e5e4ee3c3c0e9ac72561334e6eb23cc233b1bcd0a0063d...,2022-03-26,False,ok Thank for calling [CLIENT] Tech Hello? Hi. ...,march,7054,[ok Thank for calling [CLIENT] Tech Hello? Hi....
1,ae6128d8d240d28edbcb7eb27256780cf662a2be1aec8c...,2022-03-02,False,"on on Oh, My name is [NAME]. May I please have...",march,7467,"[on on Oh, My name is [NAME]. May I please hav..."
2,4899f9acdb70b0246b2d0ac2871dbcea8267e57b060154...,2022-03-12,False,Hello. We for calling [CLIENT] Tech Coach. My ...,march,3953,[Hello. We for calling [CLIENT] Tech Coach. My...
3,0589751e11caac0eac815faf84f1a366d70b86b78324a2...,2022-03-20,False,Hello. Thank you so much for contacting [CLIEN...,march,6998,[Hello. Thank you so much for contacting [CLIE...
4,6c4790cb02e4f556d73436f79268d5c99de50a9acec40e...,2022-03-15,False,Thank you very much Hello? can I can you hear ...,march,7550,[Thank you very much Hello? can I can you hear...


In [39]:
transcript_test.head()

,id,sales_offer_date,accepted_flg,encrypted_collated_transcription,month,length,transcript_chunked
1044431,4ea0f36550494fca16e9cde513b6981a0b276dc09b8dd9...,2022-07-15,False,hi Thank for calling [CLIENT] coach [CLIENT]. ...,july,19202,[hi Thank for calling [CLIENT] coach [CLIENT]....
1044432,281a94c5a1b46b125f590644613579e36304ae9cbaf75a...,2022-07-04,False,Thank you for calling [CLIENT] Tech Coach. My ...,july,8179,[Thank you for calling [CLIENT] Tech Coach. My...
1044433,a44023139d02e9c8670acbfeeee97d4e158196771d09f7...,2022-07-30,False,Hi. Thank you for calling [CLIENT] Tech Coach....,july,3655,[Hi. Thank you for calling [CLIENT] Tech Coach...
1044434,1336eca614e19d8e1540b8120062504a9d3f728609dcc1...,2022-07-21,False,Good afternoon. Thank you for calling [CLIENT]...,july,9055,[Good afternoon. Thank you for calling [CLIENT...
1044435,5188a2fd5ca86d4e9a40ea308ea7f938fce1bf06484dfa...,2022-07-06,False,Good for your amount of [CLIENT]. tech coach y...,july,2318,[Good for your amount of [CLIENT]. tech coach ...


In [40]:
transcript_train.to_csv("fe_data_v2/transcript_data/transcript_train_data_neo.csv", index = False)

In [41]:
transcript_test.to_csv("fe_data_v2/transcript_data/transcript_test_data_neo.csv", index = False)

In [42]:
mega_sentence = transcript_train[transcript_train["length"] == max(transcript_train["length"])]["encrypted_collated_transcription"]

In [43]:
mega_list = transform_sentence(transcript_train.iloc[464188, 3], 2000)
len(mega_list)

40

#### Concatenation function 

##### This is a potential solution for concatenating your summarized data

In [44]:
list_of_list = list(transcript_train["transcript_chunked"])
len(list_of_list[0])

4

In [45]:
def concatenate_sentence(chunked_list):
    summary = ""
    for i in range(len(chunked_list)):
        summary += chunked_list[i]
    return summary

In [46]:
summary_lists = []
for i in tqdm(range(len(list_of_list))):
    summary_lists.append(concatenate_sentence(list_of_list[i]))

100%|█████████████████████████████████████████████████████████████████████| 1044431/1044431 [00:05<00:00, 185289.21it/s]


In [47]:
len(summary_lists[0])

7054